In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

from sklearn.neural_network import MLPRegressor

In [ ]:
with open("training.pkl", "rb") as f:
    df = pickle.load(f)

In [ ]:
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()

In [ ]:
clf = MLPRegressor(verbose=True, hidden_layer_sizes=(50,100,100,50), max_iter=100).fit(x,y)

In [ ]:
input = [[580*2, 50*2, 0, 0, 550*2, 250*2]] # example input
size = 10000
for i in range(size):
    output = clf.predict([input[i]])
    next = [output[0][0]+input[i][0], output[0][1]+input[i][1], output[0][0], output[0][1], input[i][-2], input[i][-1]]
    input.append(next)
x = np.array(input)

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
im = Image.open("intersection2.png")
im = ImageOps.flip(im)
ax.set_xlim(0,640)
ax.set_ylim(0,360)
ax.imshow(im, origin='lower')
ax.scatter(x[:,:1]/2, x[:,1:2]/2)